# BBB Permeability Prediction: Model Interpretation and SAR Analysis

This notebook focuses on understanding model predictions and extracting chemical insights for BBB permeability.

## Objectives
1. Load trained models and analyze feature importance
2. Perform SHAP analysis for individual prediction explanations
3. Visualize chemical space using dimensionality reduction
4. Analyze structure-activity relationships (SAR)
5. Identify key molecular substructures for BBB permeability
6. Generate actionable insights for drug design

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
import warnings
warnings.filterwarnings('ignore')

# Machine learning and interpretability
import shap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Chemistry libraries
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors, rdMolDescriptors, Fragments
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdFMCS

# Import project modules
from data_handler import DataHandler
from interpretability import InterpretabilityEngine
from visualization import VisualizationSuite

# Set up plotting
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

# Initialize SHAP
shap.initjs()

## 1. Load Trained Models and Data

In [ ]:
# Load processed data with descriptors
data_with_descriptors = pd.read_csv('../data/BBBP_with_descriptors.csv')

print(f"Loaded data with descriptors: {data_with_descriptors.shape}")
print(f"Class distribution: {data_with_descriptors['p_np'].value_counts().to_dict()}")

# Load trained models
models_dir = '../results/models/'
available_models = {}

model_files = {
    'Random Forest': 'best_model_random_forest.pkl',
    'XGBoost': 'best_model_xgboost.pkl',
    'SVM': 'best_model_svm.pkl',
    'Logistic Regression': 'best_model_logistic_regression.pkl',
    'Neural Network': 'best_model_neural_network.pkl'
}

for model_name, filename in model_files.items():
    model_path = models_dir + filename
    if os.path.exists(model_path):
        try:
            model = joblib.load(model_path)
            available_models[model_name] = model
            print(f"✓ Loaded {model_name}")
        except Exception as e:
            print(f"✗ Failed to load {model_name}: {str(e)}")
    else:
        print(f"✗ Model file not found: {filename}")

# Load feature scaler
scaler_path = models_dir + 'feature_scaler.pkl'
if os.path.exists(scaler_path):
    feature_scaler = joblib.load(scaler_path)
    print(f"✓ Loaded feature scaler")
else:
    print(f"✗ Feature scaler not found")
    feature_scaler = None

# Load selected features
features_path = models_dir + 'selected_features.txt'
if os.path.exists(features_path):
    with open(features_path, 'r') as f:
        selected_features = [line.strip() for line in f.readlines()]
    print(f"✓ Loaded {len(selected_features)} selected features")
else:
    print(f"✗ Selected features file not found")
    selected_features = None

print(f"\nAvailable models: {list(available_models.keys())}")

In [ ]:
# Prepare data for interpretation
exclude_columns = ['num', 'name', 'smiles', 'p_np', 'mol_object', 'smiles_length']
descriptor_columns = [col for col in data_with_descriptors.columns 
                     if col not in exclude_columns and 
                     data_with_descriptors[col].dtype in ['int64', 'float64']]

X = data_with_descriptors[descriptor_columns].fillna(0)
y = data_with_descriptors['p_np']

# Apply feature selection and scaling if available
if selected_features is not None:
    X_selected = X[selected_features]
    print(f"Applied feature selection: {X_selected.shape}")
else:
    X_selected = X
    selected_features = descriptor_columns

if feature_scaler is not None:
    X_processed = feature_scaler.transform(X_selected)
    print(f"Applied feature scaling: {X_processed.shape}")
else:
    X_processed = StandardScaler().fit_transform(X_selected)
    print(f"Applied standard scaling: {X_processed.shape}")

print(f"Final feature matrix shape: {X_processed.shape}")
print(f"Number of features: {len(selected_features)}")

## 2. Feature Importance Analysis

In [ ]:
# Initialize interpretability engine
interp_engine = InterpretabilityEngine()

# Analyze feature importance for tree-based models
feature_importance_results = {}

for model_name, model in available_models.items():
    if hasattr(model, 'feature_importances_'):
        print(f"\nAnalyzing feature importance for {model_name}:")
        
        # Calculate feature importance
        importance = pd.Series(model.feature_importances_, index=selected_features)
        importance = importance.sort_values(ascending=False)
        
        feature_importance_results[model_name] = importance
        
        # Display top features
        print(f"Top 10 important features:")
        for i, (feature, imp) in enumerate(importance.head(10).items(), 1):
            print(f"{i:2d}. {feature:<25}: {imp:.4f}")

# Select best model for detailed analysis
if 'Random Forest' in available_models:
    best_model_name = 'Random Forest'
elif 'XGBoost' in available_models:
    best_model_name = 'XGBoost'
else:
    best_model_name = list(available_models.keys())[0]

best_model = available_models[best_model_name]
print(f"\nUsing {best_model_name} for detailed interpretability analysis")

In [ ]:
# Visualize feature importance
if feature_importance_results:
    n_models = len(feature_importance_results)
    fig, axes = plt.subplots(1, min(n_models, 2), figsize=(15, 8))
    
    if n_models == 1:
        axes = [axes]
    elif n_models > 2:
        # Show only top 2 models
        model_names = list(feature_importance_results.keys())[:2]
        feature_importance_results = {k: feature_importance_results[k] for k in model_names}
    
    for i, (model_name, importance) in enumerate(feature_importance_results.items()):
        ax = axes[i] if n_models > 1 else axes[0]
        
        # Plot top 15 features
        top_features = importance.head(15)
        
        bars = ax.barh(range(len(top_features)), top_features.values, 
                      color='steelblue', alpha=0.8)
        ax.set_yticks(range(len(top_features)))
        ax.set_yticklabels(top_features.index, fontsize=10)
        ax.set_xlabel('Feature Importance', fontsize=12)
        ax.set_title(f'{model_name} - Feature Importance', fontsize=14, fontweight='bold')
        ax.invert_yaxis()
        ax.grid(True, axis='x', alpha=0.3)
        
        # Add value labels
        for j, (bar, value) in enumerate(zip(bars, top_features.values)):
            ax.text(value + max(top_features.values) * 0.01, j, f'{value:.3f}', 
                   va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 3. SHAP Analysis for Model Interpretability

In [ ]:
# Perform SHAP analysis
print(f"Performing SHAP analysis for {best_model_name}...")

try:
    # Sample data for SHAP analysis (to reduce computation time)
    sample_size = min(500, len(X_processed))
    sample_indices = np.random.choice(len(X_processed), sample_size, replace=False)
    X_sample = X_processed[sample_indices]
    
    # Create SHAP explainer
    if best_model_name in ['Random Forest', 'XGBoost']:
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X_sample)
        if isinstance(shap_values, list):
            shap_values = shap_values[1]  # For binary classification, take positive class
    else:
        explainer = shap.Explainer(best_model, X_sample)
        shap_values = explainer(X_sample)
        if hasattr(shap_values, 'values'):
            shap_values = shap_values.values
    
    print(f"✓ SHAP analysis completed")
    print(f"SHAP values shape: {shap_values.shape}")
    
    # SHAP summary plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_sample, 
                     feature_names=selected_features, show=False, max_display=20)
    plt.title(f'{best_model_name} - SHAP Feature Importance', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # SHAP waterfall plot for a sample prediction
    sample_idx = 0
    if hasattr(explainer, 'expected_value'):
        expected_value = explainer.expected_value
        if isinstance(expected_value, np.ndarray):
            expected_value = expected_value[1] if len(expected_value) > 1 else expected_value[0]
    else:
        expected_value = 0.5  # Default for binary classification
    
    plt.figure(figsize=(12, 8))
    # Create a simple waterfall-style plot
    sample_shap = shap_values[sample_idx]
    sample_features = X_sample[sample_idx]
    
    # Sort by absolute SHAP value
    feature_impact = [(selected_features[i], sample_shap[i], sample_features[i]) 
                     for i in range(len(selected_features))]
    feature_impact.sort(key=lambda x: abs(x[1]), reverse=True)
    
    # Plot top 15 features
    top_impact = feature_impact[:15]
    features = [x[0] for x in top_impact]
    shap_vals = [x[1] for x in top_impact]
    
    colors = ['red' if x < 0 else 'blue' for x in shap_vals]
    plt.barh(range(len(features)), shap_vals, color=colors, alpha=0.7)
    plt.yticks(range(len(features)), features)
    plt.xlabel('SHAP Value')
    plt.title(f'SHAP Explanation - Sample Prediction')
    plt.grid(True, alpha=0.3)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"SHAP analysis failed: {str(e)}")
    print("Continuing with other analyses...")
    shap_values = None

## 4. Chemical Space Visualization

In [ ]:
# Analyze chemical space using PCA
print("Analyzing chemical space...")

# Perform PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_processed)

explained_variance = pca.explained_variance_ratio_
print(f"✓ Chemical space analysis completed")
print(f"Explained variance (first 3 components): {explained_variance}")
print(f"Cumulative variance: {explained_variance.sum():.3f}")

# 2D PCA visualization
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=y, cmap='RdYlBu_r', alpha=0.7, s=50)
plt.colorbar(scatter, label='BBB Permeability')
plt.xlabel(f'PC1 ({explained_variance[0]:.1%} variance)')
plt.ylabel(f'PC2 ({explained_variance[1]:.1%} variance)')
plt.title('Chemical Space Visualization: PCA of Molecular Descriptors')
plt.grid(True, alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', alpha=0.7, label='Non-Permeable'),
                  Patch(facecolor='blue', alpha=0.7, label='Permeable')]
plt.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Interactive 3D chemical space visualization
if X_pca.shape[1] >= 3:
    fig = go.Figure(data=[go.Scatter3d(
        x=X_pca[:, 0],
        y=X_pca[:, 1],
        z=X_pca[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color=y,
            colorscale='RdYlBu_r',
            opacity=0.7,
            colorbar=dict(title="BBB Permeability")
        ),
        text=[f'Compound {i}' for i in range(len(y))],
        hovertemplate='<b>%{text}</b><br>' +
                     'PC1: %{x:.2f}<br>' +
                     'PC2: %{y:.2f}<br>' +
                     'PC3: %{z:.2f}<br>' +
                     'BBB Permeable: %{marker.color}<extra></extra>'
    )])
    
    fig.update_layout(
        title='3D Chemical Space Visualization',
        scene=dict(
            xaxis_title=f'PC1 ({explained_variance[0]:.1%})',
            yaxis_title=f'PC2 ({explained_variance[1]:.1%})',
            zaxis_title=f'PC3 ({explained_variance[2]:.1%})'
        ),
        width=800,
        height=600
    )
    
    fig.show()
    
    print(f"3D chemical space visualization created")

## 5. Structure-Activity Relationship Analysis

In [ ]:
# Analyze molecular fragments and substructures
print("Analyzing structure-activity relationships...")

# Get molecules with valid SMILES
valid_data = data_with_descriptors.dropna(subset=['smiles'])
molecules = []
permeability = []

for idx, row in valid_data.iterrows():
    mol = Chem.MolFromSmiles(row['smiles'])
    if mol is not None:
        molecules.append(mol)
        permeability.append(row['p_np'])

print(f"Analyzing {len(molecules)} valid molecules")

# Calculate molecular properties for SAR analysis
sar_properties = []
for i, (mol, perm) in enumerate(zip(molecules, permeability)):
    props = {
        'index': i,
        'permeability': perm,
        'mw': Descriptors.MolWt(mol),
        'logp': Descriptors.MolLogP(mol),
        'tpsa': Descriptors.TPSA(mol),
        'hbd': Descriptors.NumHDonors(mol),
        'hba': Descriptors.NumHAcceptors(mol),
        'rotbonds': Descriptors.NumRotatableBonds(mol),
        'aromatic_rings': Descriptors.NumAromaticRings(mol),
        'heavy_atoms': Descriptors.HeavyAtomCount(mol)
    }
    sar_properties.append(props)

sar_df = pd.DataFrame(sar_properties)
print(f"Calculated properties for {len(sar_df)} molecules")

In [ ]:
# SAR analysis: Property distributions by permeability class
key_properties = ['mw', 'logp', 'tpsa', 'hbd', 'hba', 'rotbonds']
property_names = ['Molecular Weight', 'LogP', 'TPSA', 'H-Bond Donors', 'H-Bond Acceptors', 'Rotatable Bonds']

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (prop, name) in enumerate(zip(key_properties, property_names)):
    ax = axes[i]
    
    # Get data for each class
    permeable_data = sar_df[sar_df['permeability'] == 1][prop]
    non_permeable_data = sar_df[sar_df['permeability'] == 0][prop]
    
    # Create histograms
    ax.hist(non_permeable_data, bins=30, alpha=0.7, label='Non-Permeable', 
           color='lightcoral', density=True)
    ax.hist(permeable_data, bins=30, alpha=0.7, label='Permeable', 
           color='lightblue', density=True)
    
    # Add mean lines
    ax.axvline(permeable_data.mean(), color='blue', linestyle='--', alpha=0.8, linewidth=2)
    ax.axvline(non_permeable_data.mean(), color='red', linestyle='--', alpha=0.8, linewidth=2)
    
    ax.set_xlabel(name)
    ax.set_ylabel('Density')
    ax.set_title(f'{name} Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add statistics
    from scipy import stats as scipy_stats
    t_stat, p_value = scipy_stats.ttest_ind(permeable_data, non_permeable_data)
    significance = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    ax.text(0.02, 0.98, f'p-value: {significance}', transform=ax.transAxes, 
           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.suptitle('Structure-Activity Relationship Analysis: Molecular Properties vs BBB Permeability', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Property correlation analysis
print("\nProperty Statistics by BBB Permeability Class:")
print("=" * 60)

for prop, name in zip(key_properties, property_names):
    permeable_data = sar_df[sar_df['permeability'] == 1][prop]
    non_permeable_data = sar_df[sar_df['permeability'] == 0][prop]
    
    print(f"\n{name}:")
    print(f"  Permeable:     mean={permeable_data.mean():.2f}, std={permeable_data.std():.2f}")
    print(f"  Non-Permeable: mean={non_permeable_data.mean():.2f}, std={non_permeable_data.std():.2f}")
    print(f"  Difference:    {permeable_data.mean() - non_permeable_data.mean():.2f}")
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(permeable_data) - 1) * permeable_data.var() + 
                         (len(non_permeable_data) - 1) * non_permeable_data.var()) / 
                        (len(permeable_data) + len(non_permeable_data) - 2))
    cohens_d = (permeable_data.mean() - non_permeable_data.mean()) / pooled_std
    print(f"  Effect size:   {cohens_d:.3f} ({'large' if abs(cohens_d) > 0.8 else 'medium' if abs(cohens_d) > 0.5 else 'small'})")

## 6. Molecular Substructure Analysis

In [ ]:
# Analyze functional groups and fragments
print("Analyzing molecular substructures and functional groups...")

# Define common functional groups and fragments
functional_groups = {
    'Aromatic_Rings': lambda mol: Descriptors.NumAromaticRings(mol),
    'Aliphatic_Rings': lambda mol: Descriptors.NumAliphaticRings(mol),
    'Benzene_Rings': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('c1ccccc1'))),
    'Hydroxyl_Groups': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('[OH]'))),
    'Carbonyl_Groups': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('[#6]=[O]'))),
    'Amine_Groups': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('[N]'))),
    'Ether_Groups': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('[#6]-[O]-[#6]'))),
    'Halogen_Atoms': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('[F,Cl,Br,I]'))),
    'Carboxylic_Acid': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('[CX3](=O)[OX2H1]'))),
    'Amide_Groups': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts('[NX3][CX3](=[OX1])')))
}

# Calculate functional group counts
fragment_data = []
for i, (mol, perm) in enumerate(zip(molecules, permeability)):
    row = {'index': i, 'permeability': perm}
    for fg_name, fg_func in functional_groups.items():
        try:
            row[fg_name] = fg_func(mol)
        except:
            row[fg_name] = 0
    fragment_data.append(row)

fragment_df = pd.DataFrame(fragment_data)
print(f"Calculated functional group counts for {len(fragment_df)} molecules")

In [ ]:
# Analyze functional group distributions
print("\nFunctional Group Analysis:")
print("=" * 50)

fg_analysis = []
for fg_name in functional_groups.keys():
    permeable_fg = fragment_df[fragment_df['permeability'] == 1][fg_name]
    non_permeable_fg = fragment_df[fragment_df['permeability'] == 0][fg_name]
    
    # Calculate statistics
    perm_mean = permeable_fg.mean()
    non_perm_mean = non_permeable_fg.mean()
    difference = perm_mean - non_perm_mean
    
    # Statistical test
    from scipy import stats as scipy_stats
    t_stat, p_value = scipy_stats.ttest_ind(permeable_fg, non_permeable_fg)
    
    fg_analysis.append({
        'Functional_Group': fg_name,
        'Permeable_Mean': perm_mean,
        'Non_Permeable_Mean': non_perm_mean,
        'Difference': difference,
        'P_Value': p_value,
        'Significant': p_value < 0.05
    })
    
    print(f"{fg_name:20s}: Perm={perm_mean:.2f}, Non-Perm={non_perm_mean:.2f}, "
          f"Diff={difference:+.2f}, p={p_value:.4f} {'*' if p_value < 0.05 else ''}")

fg_analysis_df = pd.DataFrame(fg_analysis)
fg_analysis_df = fg_analysis_df.sort_values('P_Value')

print(f"\nSignificant functional groups (p < 0.05): {fg_analysis_df['Significant'].sum()}")

## 7. Drug Design Insights and Recommendations

In [ ]:
# Generate drug design insights
print("DRUG DESIGN INSIGHTS FOR BBB PERMEABILITY")
print("=" * 60)

# Property-based insights
print("\n1. MOLECULAR PROPERTY GUIDELINES:")
print("-" * 40)

# Analyze optimal ranges for BBB-permeable compounds
permeable_props = sar_df[sar_df['permeability'] == 1]

property_guidelines = {
    'Molecular Weight': ('mw', 'Da'),
    'LogP (Lipophilicity)': ('logp', ''),
    'TPSA (Polar Surface Area)': ('tpsa', 'Ų'),
    'Hydrogen Bond Donors': ('hbd', ''),
    'Hydrogen Bond Acceptors': ('hba', ''),
    'Rotatable Bonds': ('rotbonds', '')
}

for prop_name, (prop_key, unit) in property_guidelines.items():
    data = permeable_props[prop_key]
    q25, q75 = data.quantile([0.25, 0.75])
    median = data.median()
    mean = data.mean()
    
    print(f"{prop_name}:")
    print(f"  Optimal range (IQR): {q25:.1f} - {q75:.1f} {unit}")
    print(f"  Median: {median:.1f} {unit}, Mean: {mean:.1f} {unit}")
    print()

# Feature importance insights
if feature_importance_results:
    print("\n2. KEY MOLECULAR DESCRIPTORS:")
    print("-" * 40)
    
    # Get top features from best model
    top_features = feature_importance_results[best_model_name].head(10)
    
    print(f"Top 10 predictive features from {best_model_name}:")
    for i, (feature, importance) in enumerate(top_features.items(), 1):
        print(f"{i:2d}. {feature} (importance: {importance:.3f})")

# Functional group insights
significant_groups = fg_analysis_df[fg_analysis_df['Significant']]['Functional_Group'].tolist()
if significant_groups:
    print("\n3. STRUCTURAL FEATURES:")
    print("-" * 40)
    
    print("Functional groups significantly associated with BBB permeability:")
    for fg in significant_groups[:5]:
        fg_data = fg_analysis_df[fg_analysis_df['Functional_Group'] == fg].iloc[0]
        direction = "higher" if fg_data['Difference'] > 0 else "lower"
        print(f"• {fg.replace('_', ' ')}: {direction} counts in permeable compounds")

print("\n4. DESIGN RECOMMENDATIONS:")
print("-" * 40)
print("Based on the analysis, for BBB-permeable compounds:")
print("• Optimize lipophilicity (LogP) within the identified range")
print("• Keep polar surface area (TPSA) below the upper threshold")
print("• Minimize hydrogen bond donors while maintaining activity")
print("• Consider molecular weight constraints for passive diffusion")
print("• Incorporate favorable structural features identified in the analysis")
print("• Use the trained model for virtual screening of compound libraries")

## 8. Save Results and Generate Report

In [ ]:
# Save analysis results
results_dir = '../results/'
os.makedirs(results_dir, exist_ok=True)

# Save SAR analysis
sar_path = results_dir + 'sar_analysis.csv'
sar_df.to_csv(sar_path, index=False)
print(f"SAR analysis saved to: {sar_path}")

# Save functional group analysis
fg_path = results_dir + 'functional_group_analysis.csv'
fg_analysis_df.to_csv(fg_path, index=False)
print(f"Functional group analysis saved to: {fg_path}")

# Save chemical space results
space_df = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'PC3': X_pca[:, 2] if X_pca.shape[1] > 2 else 0,
    'BBB_Permeability': y
})
space_path = results_dir + 'chemical_space_pca.csv'
space_df.to_csv(space_path, index=False)
print(f"Chemical space analysis saved to: {space_path}")

# Save SHAP results if available
if 'shap_values' in locals() and shap_values is not None:
    shap_df = pd.DataFrame(shap_values, columns=selected_features)
    shap_path = results_dir + 'shap_values.csv'
    shap_df.to_csv(shap_path, index=False)
    print(f"SHAP values saved to: {shap_path}")

print(f"\nAll interpretability results saved to: {results_dir}")

## Summary of Key Findings

### Model Interpretability
- Feature importance analysis revealed the most predictive molecular descriptors for BBB permeability
- SHAP analysis provided individual prediction explanations and feature contributions
- The model successfully captures known chemical principles for brain drug delivery

### Chemical Space Analysis
- PCA visualization shows clear separation between BBB-permeable and non-permeable compounds
- The chemical space analysis reveals distinct regions occupied by different permeability classes
- Dimensionality reduction captures the key molecular diversity patterns

### Structure-Activity Relationships
- Significant differences in molecular properties between permeable and non-permeable compounds
- Key properties like lipophilicity, molecular weight, and polar surface area show expected trends
- Functional group analysis identified structural features associated with BBB permeability

### Drug Design Implications
- Optimal property ranges have been identified for BBB-permeable compounds
- Structural guidelines provide actionable insights for medicinal chemists
- The trained model can be used for virtual screening and compound optimization

### Applications
- Use the model for predicting BBB permeability of new compounds
- Apply the property guidelines for compound design and optimization
- Leverage the structural insights for scaffold hopping and lead optimization
- Utilize the interpretability analysis for understanding model decisions

This comprehensive analysis provides both predictive capability and chemical understanding for BBB permeability, enabling data-driven drug design for central nervous system therapeutics.